In [1]:
import itertools as it
import os

from cliffs_delta import cliffs_delta
from iterpop import iterpop as ip
import matplotlib as mpl
from matplotlib import pyplot as plt
from nbmetalog import nbmetalog as nbm
import numpy as np
import pandas as pd
from scipy import stats as scipy_stats
import seaborn as sns
from teeplot import teeplot as tp

from pylib import describe_effect


In [2]:
# prints metadata about notebook runtime
nbm.print_metadata()


context: ci
hostname: fv-az1052-515
interpreter: 3.10.12 (main, Sep 11 2024, 15:47:36) [GCC 11.4.0]
nbcellexec: null
nbname: spatial-nuissance-phylometrics
nbpath: /home/runner/work/hstrat-evolutionary-inference/hstrat-evolutionary-inference/binder/avida-individual/spatial-nuissance-phylometrics.ipynb
revision: null
session: e50ea923-8a7a-4c15-a58e-434a4a7cd491
timestamp: 2024-11-21T04:53:12Z00:00


IPython==8.13.0
keyname==0.5.2
yaml==6.0
iterpop==0.4.1
matplotlib==3.7.1
nbmetalog==0.2.6
numpy==1.21.6
pandas==1.3.5
scipy==1.7.3
seaborn==0.12.1
teeplot==1.0.1
re==2.2.1
comm==0.2.2
logging==0.5.1.2
psutil==5.9.8
zmq==24.0.1
json==2.0.9
ipykernel==6.29.3


In [3]:
df = pd.read_csv("https://osf.io/djevp/download")
dfdigest = np.bitwise_xor.reduce(
    pd.util.hash_pandas_object(df),
)
print("{:x}".format(dfdigest))
df


877c5e50c2c17e00


,a,epoch,mut_distn,treatment,seed,phylogeny-snapshot-100000.csv,_,colless_like_index,average_depth,average_origin_time,...,variance_pairwise_distance_branch_only,mrca_depth,sackin_index,mean_evolutionary_distinctiveness,sum_evolutionary_distinctiveness,variance_evolutionary_distinctiveness,resolution,subsampling-fraction,trie-postprocess,ext
0,collapsed-phylogeny,0,default,individual_selection_weak,166,NaN,/mnt/home/mmore500/scratch/data/hstrat-evoluti...,16413.377699,inf,99903.330314,...,29.762505,1,37604,116.809311,319823.893642,12111.310369,NaN,NaN,NaN,NaN
1,collapsed-phylogeny,0,default,spatial_plain,285,NaN,/mnt/home/mmore500/scratch/data/hstrat-evoluti...,5889.044246,inf,99860.938076,...,17.246662,1,19441,169.082623,314155.514230,9583.662151,NaN,NaN,NaN,NaN
2,collapsed-phylogeny,0,default,individual_ecology_weak,97,NaN,/mnt/home/mmore500/scratch/data/hstrat-evoluti...,24328.758494,inf,99949.740000,...,26.685759,1,44783,42.032856,113404.644817,624.827781,NaN,NaN,NaN,NaN
3,collapsed-phylogeny,0,default,individual_spatial_ecology_rich,202,NaN,/mnt/home/mmore500/scratch/data/hstrat-evoluti...,13872.550476,inf,99831.008690,...,25.321388,1,36795,257.400153,719433.428105,12815.294360,NaN,NaN,NaN,NaN
4,collapsed-phylogeny,0,default,selection_strong,134,NaN,/mnt/home/mmore500/scratch/data/hstrat-evoluti...,11151.009191,inf,99882.926099,...,27.690557,1,28725,146.499653,337095.700472,11015.743172,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2395,reconstructed-tree,0,default,individual_selection_weak,174,NaN,/mnt/home/mmore500/scratch/data/hstrat-evoluti...,13068.933629,22.807692,99948.311406,...,24.558595,1,28133,55.940519,125083.000000,932.185874,10.0,1.0,naive,.csv.gz
2396,reconstructed-tree,0,default,individual_selection_weak,155,NaN,/mnt/home/mmore500/scratch/data/hstrat-evoluti...,16115.370601,25.311671,99910.315652,...,30.149655,1,31841,125.896994,284779.000000,6755.799258,100.0,1.0,naive,.csv.gz
2397,reconstructed-tree,0,default,individual_selection_weak,151,NaN,/mnt/home/mmore500/scratch/data/hstrat-evoluti...,15862.794784,24.893363,99913.065014,...,26.904952,1,31043,120.667920,272709.500000,4576.502673,33.0,1.0,naive,.csv.gz
2398,reconstructed-tree,0,default,individual_spatial_ecology_some,221,NaN,/mnt/home/mmore500/scratch/data/hstrat-evoluti...,14001.023216,24.323682,99911.224317,...,25.275680,1,30394,134.458081,311136.000000,6716.805266,10.0,1.0,naive,.csv.gz


In [4]:
# phylometrics whittled down to descriptive, non-redundant set
phylometrics = [
    #     'diversity',
    "colless_like_index",
    #     'average_depth',
    #     'average_origin_time',
    #     'average_origin_time_normalized',
    #     'max_depth',
    "mean_pairwise_distance",
    #     'mean_pairwise_distance_branch_only',
    #     'num_active',
    # "num_ancestors",
    "sum_distance",
    #     'num_outside',
    #     'num_roots',
    #     'num_taxa',
    #     'phylogenetic_diversity',
    #     'sum_pairwise_distance',
    #     'sum_pairwise_distance_branch_only',
    #     'total_orgs',
    #     'tree_size',
    #     'variance_pairwise_distance',
    #     'variance_pairwise_distance_branch_only',
    #     'mrca_depth',
    #     'sackin_index',
    "mean_evolutionary_distinctiveness",
    #     'sum_evolutionary_distinctiveness',
    #     'variance_evolutionary_distinctiveness',
]


In [5]:
methodological_variables = [
    "resolution",
]


In [6]:
evolutionary_variables = [
]


In [7]:
sensitivity_analysis_variables = [
    "epoch",
    "mut_distn",
]


In [8]:
fixed_variables = [
    "subsampling-fraction",  # only use 1.0
    "trie-postprocess",  # just use naive postprocessing for now
]


In [9]:
# ensure fixed variables
df = df[
    ((df["subsampling-fraction"] == 1.0) | df["subsampling-fraction"].isna())
    & ((df["trie-postprocess"] == "naive") | df["trie-postprocess"].isna())
].copy()

for fixed_variable in fixed_variables:
    assert len(df[fixed_variable].dropna().unique()) == 1

df


,a,epoch,mut_distn,treatment,seed,phylogeny-snapshot-100000.csv,_,colless_like_index,average_depth,average_origin_time,...,variance_pairwise_distance_branch_only,mrca_depth,sackin_index,mean_evolutionary_distinctiveness,sum_evolutionary_distinctiveness,variance_evolutionary_distinctiveness,resolution,subsampling-fraction,trie-postprocess,ext
0,collapsed-phylogeny,0,default,individual_selection_weak,166,NaN,/mnt/home/mmore500/scratch/data/hstrat-evoluti...,16413.377699,inf,99903.330314,...,29.762505,1,37604,116.809311,319823.893642,12111.310369,NaN,NaN,NaN,NaN
1,collapsed-phylogeny,0,default,spatial_plain,285,NaN,/mnt/home/mmore500/scratch/data/hstrat-evoluti...,5889.044246,inf,99860.938076,...,17.246662,1,19441,169.082623,314155.514230,9583.662151,NaN,NaN,NaN,NaN
2,collapsed-phylogeny,0,default,individual_ecology_weak,97,NaN,/mnt/home/mmore500/scratch/data/hstrat-evoluti...,24328.758494,inf,99949.740000,...,26.685759,1,44783,42.032856,113404.644817,624.827781,NaN,NaN,NaN,NaN
3,collapsed-phylogeny,0,default,individual_spatial_ecology_rich,202,NaN,/mnt/home/mmore500/scratch/data/hstrat-evoluti...,13872.550476,inf,99831.008690,...,25.321388,1,36795,257.400153,719433.428105,12815.294360,NaN,NaN,NaN,NaN
4,collapsed-phylogeny,0,default,selection_strong,134,NaN,/mnt/home/mmore500/scratch/data/hstrat-evoluti...,11151.009191,inf,99882.926099,...,27.690557,1,28725,146.499653,337095.700472,11015.743172,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2395,reconstructed-tree,0,default,individual_selection_weak,174,NaN,/mnt/home/mmore500/scratch/data/hstrat-evoluti...,13068.933629,22.807692,99948.311406,...,24.558595,1,28133,55.940519,125083.000000,932.185874,10.0,1.0,naive,.csv.gz
2396,reconstructed-tree,0,default,individual_selection_weak,155,NaN,/mnt/home/mmore500/scratch/data/hstrat-evoluti...,16115.370601,25.311671,99910.315652,...,30.149655,1,31841,125.896994,284779.000000,6755.799258,100.0,1.0,naive,.csv.gz
2397,reconstructed-tree,0,default,individual_selection_weak,151,NaN,/mnt/home/mmore500/scratch/data/hstrat-evoluti...,15862.794784,24.893363,99913.065014,...,26.904952,1,31043,120.667920,272709.500000,4576.502673,33.0,1.0,naive,.csv.gz
2398,reconstructed-tree,0,default,individual_spatial_ecology_some,221,NaN,/mnt/home/mmore500/scratch/data/hstrat-evoluti...,14001.023216,24.323682,99911.224317,...,25.275680,1,30394,134.458081,311136.000000,6716.805266,10.0,1.0,naive,.csv.gz


In [10]:
# exclude extraneous variable values
df = df[(df["a"] != "consolidated-phylogeny")].copy()

df


,a,epoch,mut_distn,treatment,seed,phylogeny-snapshot-100000.csv,_,colless_like_index,average_depth,average_origin_time,...,variance_pairwise_distance_branch_only,mrca_depth,sackin_index,mean_evolutionary_distinctiveness,sum_evolutionary_distinctiveness,variance_evolutionary_distinctiveness,resolution,subsampling-fraction,trie-postprocess,ext
0,collapsed-phylogeny,0,default,individual_selection_weak,166,NaN,/mnt/home/mmore500/scratch/data/hstrat-evoluti...,16413.377699,inf,99903.330314,...,29.762505,1,37604,116.809311,319823.893642,12111.310369,NaN,NaN,NaN,NaN
1,collapsed-phylogeny,0,default,spatial_plain,285,NaN,/mnt/home/mmore500/scratch/data/hstrat-evoluti...,5889.044246,inf,99860.938076,...,17.246662,1,19441,169.082623,314155.514230,9583.662151,NaN,NaN,NaN,NaN
2,collapsed-phylogeny,0,default,individual_ecology_weak,97,NaN,/mnt/home/mmore500/scratch/data/hstrat-evoluti...,24328.758494,inf,99949.740000,...,26.685759,1,44783,42.032856,113404.644817,624.827781,NaN,NaN,NaN,NaN
3,collapsed-phylogeny,0,default,individual_spatial_ecology_rich,202,NaN,/mnt/home/mmore500/scratch/data/hstrat-evoluti...,13872.550476,inf,99831.008690,...,25.321388,1,36795,257.400153,719433.428105,12815.294360,NaN,NaN,NaN,NaN
4,collapsed-phylogeny,0,default,selection_strong,134,NaN,/mnt/home/mmore500/scratch/data/hstrat-evoluti...,11151.009191,inf,99882.926099,...,27.690557,1,28725,146.499653,337095.700472,11015.743172,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2395,reconstructed-tree,0,default,individual_selection_weak,174,NaN,/mnt/home/mmore500/scratch/data/hstrat-evoluti...,13068.933629,22.807692,99948.311406,...,24.558595,1,28133,55.940519,125083.000000,932.185874,10.0,1.0,naive,.csv.gz
2396,reconstructed-tree,0,default,individual_selection_weak,155,NaN,/mnt/home/mmore500/scratch/data/hstrat-evoluti...,16115.370601,25.311671,99910.315652,...,30.149655,1,31841,125.896994,284779.000000,6755.799258,100.0,1.0,naive,.csv.gz
2397,reconstructed-tree,0,default,individual_selection_weak,151,NaN,/mnt/home/mmore500/scratch/data/hstrat-evoluti...,15862.794784,24.893363,99913.065014,...,26.904952,1,31043,120.667920,272709.500000,4576.502673,33.0,1.0,naive,.csv.gz
2398,reconstructed-tree,0,default,individual_spatial_ecology_some,221,NaN,/mnt/home/mmore500/scratch/data/hstrat-evoluti...,14001.023216,24.323682,99911.224317,...,25.275680,1,30394,134.458081,311136.000000,6716.805266,10.0,1.0,naive,.csv.gz


In [11]:
df["a"].unique()


array(['collapsed-phylogeny', 'reconstructed-tree'], dtype=object)

In [12]:
df["regime"] = df["treatment"].map(
    {
        "individual_plain": "X plain",
        "individual_selection_weak": "X weak selection",
        "individual_selection_strong": "X strong selection",
        "individual_spatial_plain": "plain",
        "individual_ecology_some": "X ecology",
        "individual_ecology_rich": "X rich ecology",
        "individual_spatial_ecology_rich": "rich ecology",
        "individual_spatial_ecology_some": "ecology",
        "individual_ecology_weak": "X weak ecology",
        "individual_spatial_ecology_weak": "weak ecology",
    },
)
df["regime"].unique()


array(['X weak selection', nan, 'X weak ecology', 'rich ecology', 'plain',
       'weak ecology', 'X rich ecology', 'X plain', 'X ecology',
       'ecology', 'X strong selection'], dtype=object)

In [13]:
df["quality"] = df.apply(
    lambda row: {
        ("reconstructed-tree", 3.0): "33% resolution",
        ("reconstructed-tree", 10.0): "10% resolution",
        ("reconstructed-tree", 33.0): "3% resolution",
        ("reconstructed-tree", 100.0): "1% resolution",
        ("collapsed-phylogeny", 0): "perfect resolution",
    }.get(
        tuple(row[["a", "resolution"]].fillna(0)),
        np.nan,  # default
    ),
    axis="columns",
)
df = df.dropna(axis="index", subset=["quality"]).copy()
df["quality"].unique()


array(['perfect resolution', '1% resolution', '10% resolution',
       '3% resolution', '33% resolution'], dtype=object)

In [14]:
df[df["a"] == "collapsed-phylogeny"]["regime"].unique()


array(['X weak selection', nan, 'X weak ecology', 'rich ecology', 'plain',
       'weak ecology', 'X rich ecology', 'X plain', 'X ecology',
       'ecology', 'X strong selection'], dtype=object)

# perfect phylogenetic data


In [15]:
melted_df = df[(df["a"] == "collapsed-phylogeny")].melt(
    id_vars=evolutionary_variables
    + methodological_variables
    + sensitivity_analysis_variables
    + ["regime"],
    value_vars=phylometrics,
)
melted_df["phylometric"] = melted_df["variable"]
melted_df


,resolution,epoch,mut_distn,regime,variable,value,phylometric
0,NaN,0,default,X weak selection,colless_like_index,16413.377699,colless_like_index
1,NaN,0,default,NaN,colless_like_index,5889.044246,colless_like_index
2,NaN,0,default,X weak ecology,colless_like_index,24328.758494,colless_like_index
3,NaN,0,default,rich ecology,colless_like_index,13872.550476,colless_like_index
4,NaN,0,default,NaN,colless_like_index,11151.009191,colless_like_index
...,...,...,...,...,...,...,...
1915,NaN,0,default,weak ecology,mean_evolutionary_distinctiveness,162.204685,mean_evolutionary_distinctiveness
1916,NaN,0,default,NaN,mean_evolutionary_distinctiveness,128.176179,mean_evolutionary_distinctiveness
1917,NaN,0,default,rich ecology,mean_evolutionary_distinctiveness,228.583965,mean_evolutionary_distinctiveness
1918,NaN,0,default,X strong selection,mean_evolutionary_distinctiveness,106.106423,mean_evolutionary_distinctiveness


# stats: kruskal wallace among regimes


In [16]:
records = []
for phylometric in phylometrics:
    for group, group_df in df[
        (df["a"] == "collapsed-phylogeny")
        & df["regime"].isin(
        [
            "plain",
            "weak ecology",
            "ecology",
            "rich ecology",
        ]
        )
    ].groupby(
        sensitivity_analysis_variables,
        as_index=False,
    ):
        kw_result, n = scipy_stats.kruskal(
            *(
                quality_series
                for quality, quality_series in group_df.groupby(["regime"])[
                    phylometric
                ]
            )
        ), min(group_df.groupby(["regime"]).count()["a"])
        records.append(
            {
                **{
                    "n": n,
                    "N": len([*group_df.groupby(["regime"])]),
                    "phylometric": phylometric,
                },
                **dict(zip(["statistic", "p"], kw_result)),
                **dict(zip(sensitivity_analysis_variables, group)),
            }
        )

with pd.option_context("display.max_colwidth", None, "display.max_rows", None):
    out_df = (
        pd.DataFrame.from_records(records)
        .sort_values(
            sensitivity_analysis_variables,
        )
        .reset_index(drop=True)
    )
    display(out_df)
    os.makedirs("outdata", exist_ok=True)
    out_df.to_csv(
        "outdata/a=phylostatistics-comparison-between-regimes-spatial-nuisance-kwallis+ext=.csv",
        index=False,
    )


,n,N,phylometric,statistic,p,epoch,mut_distn
0,30,4,colless_like_index,15.223196,1.635513e-03,0,default
1,30,4,mean_pairwise_distance,20.933168,1.086947e-04,0,default
2,30,4,sum_distance,56.421102,3.415676e-12,0,default
3,30,4,mean_evolutionary_distinctiveness,59.557741,7.306790e-13,0,default


In [17]:
records = []
for phylometric in phylometrics:
    for group, group_df in df[
        (df["a"] == "collapsed-phylogeny")
        & df["regime"].isin(
        [
            "plain",
            "weak ecology",
            "ecology",
            "rich ecology",
        ],
        )
    ].groupby(
        sensitivity_analysis_variables, as_index=False
    ):
        for regime1, regime2 in it.combinations(
            group_df["regime"].unique(), 2
        ):
            kw_result, n = scipy_stats.wilcoxon(
                *zip(
                    *zip(
                        group_df[group_df["regime"] == regime1][phylometric],
                        group_df[group_df["regime"] == regime2][phylometric],
                    )
                ),
            ), min(group_df.groupby(["regime"]).count()["a"])
            records.append(
                {
                    **{
                        "regime1": regime1,
                        "regime2": regime2,
                        "n": n,
                        "phylometric": phylometric,
                    },
                    **dict(zip(["statistic", "p"], kw_result)),
                    **dict(zip(sensitivity_analysis_variables, group)),
                }
            )

with pd.option_context("display.max_colwidth", None, "display.max_rows", None):
    out_df = (
        pd.DataFrame.from_records(records)
        .sort_values(
            [*sensitivity_analysis_variables, "regime1", "regime2"],
        )
        .reset_index(drop=True)
    )
    display(out_df)
    os.makedirs("outdata", exist_ok=True)
    out_df.to_csv(
        "outdata/a=phylostatistics-comparison-between-regimes-allpairs-spatial-nuisance-wilcox+ext=.csv",
        index=False,
    )


,regime1,regime2,n,phylometric,statistic,p,epoch,mut_distn
0,plain,ecology,30,colless_like_index,154.0,0.106394,0,default
1,plain,ecology,30,mean_pairwise_distance,214.0,0.703564,0,default
2,plain,ecology,30,sum_distance,194.0,0.428430,0,default
3,plain,ecology,30,mean_evolutionary_distinctiveness,162.0,0.147040,0,default
4,plain,weak ecology,30,colless_like_index,226.0,0.893644,0,default
5,plain,weak ecology,30,mean_pairwise_distance,133.0,0.040702,0,default
6,plain,weak ecology,30,sum_distance,181.0,0.289477,0,default
7,plain,weak ecology,30,mean_evolutionary_distinctiveness,184.0,0.318491,0,default
8,rich ecology,ecology,30,colless_like_index,119.0,0.019569,0,default
9,rich ecology,ecology,30,mean_pairwise_distance,57.0,0.000306,0,default
